In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Feature Engineering: previous_application**

A Feature Engineering é um processo fundamental para ciência de dados. Ela se refere ao processo de criação e transformação de variáveis para melhorar o desempenho dos modelos de Machine Learning.
<br>
<br>
Nesse notebook vamos trabalhar na criação de variáveis para o dataset **`previous_application`**, fazer o join com as variáveis criadas a partir dos datasets **`POS_CASH_balance`**, **`installments_payments`** e **`credit_card_balance`** e no final deixar tudo na granularidade indivíduo através do ID **`SK_ID_CURR`**. Os dados contém informações sobre aplicações de empréstimo anteriores de um cliente na PoD Bank.
<br>
<br>
A partir desses dados podemos gerar insights sobre o histórico financeiro do cliente, incluindo seu comportamento de pagamento passado, o montante dos empréstimos anteriores, e a frequência de empréstimos. Essas informações ajudam a avaliar o risco de inadimplência futuro e a tomar decisões mais informadas ao conceder crédito.
<br>
<br>
Por se tratar de criação de variáveis, nessa etapa geralmente trabalhamos com muito volume de dados e grande demanda de processamento, por essa razão iremos utilizar o **Spark** e **Spark.SQL** para essa etapa.

<br>

## **Pacotes e Bibliotecas**

In [2]:
# Instalando o PySpark.
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# Importando bibliotecas.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
from math import ceil
import os
import sys

In [4]:
# Configurando a sessão Spark.
spark = SparkSession.builder \
    .appName('app_spark_2') \
    .config('spark.executor.memory', '4g') \
    .config('spark.driver.memory', '4g') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

<br>
<br>
<br>
<br>

## **Funções**

In [5]:
# Função para substituir espaço por underline e colocar a palavra em letras maiúsculas.
import re

def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()

    return palavra_transformada

<br>
<br>
<br>
<br>

## **Leitura e visualização do conjunto de dados**

In [6]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
dados = spark.read.csv('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/database/previous_application.csv', header=True, inferSchema=True)
dados.createOrReplaceTempView('dados')

# Mostra os dados.
dados.show()

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-------------------+---------------------+------------------------+----------------------+--------------------+-------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+----------------+--------------------+-----------+----------------+--------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|  RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURPOSE

In [7]:
# Checando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
num_rows = dados.count()

# Quantidade de colunas.
num_columns = len(dados.columns)

# Quantidade de IDs únicos.
num_id_curr = dados.select(countDistinct('SK_ID_CURR').alias('num_id_curr')).collect()[0]['num_id_curr']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_CURR" únicos do DataFrame: {num_id_curr}')

Quantidade de linhas do DataFrame: 1670214
Quantidade de colunas do DataFrame: 37
Quantidade de "SK_ID_CURR" únicos do DataFrame: 338857


In [8]:
# Exibindo a estrutura do schema do DataFrame.
dados.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_APPLICATION: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_DOWN_PAYMENT: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- WEEKDAY_APPR_PROCESS_START: string (nullable = true)
 |-- HOUR_APPR_PROCESS_START: integer (nullable = true)
 |-- FLAG_LAST_APPL_PER_CONTRACT: string (nullable = true)
 |-- NFLAG_LAST_APPL_IN_DAY: integer (nullable = true)
 |-- RATE_DOWN_PAYMENT: double (nullable = true)
 |-- RATE_INTEREST_PRIMARY: double (nullable = true)
 |-- RATE_INTEREST_PRIVILEGED: double (nullable = true)
 |-- NAME_CASH_LOAN_PURPOSE: string (nullable = true)
 |-- NAME_CONTRACT_STATUS: string (nullable = true)
 |-- DAYS_DECISION: integer (nullable = true)
 |-- NAME_PAYMENT_TYPE: string (nullable = true)
 |-- CODE_REJECT_REASON: string (nullable = t

In [9]:
# Extraindo os valores únicos da coluna 'NAME_CONTRACT_STATUS'.
status = dados.select('NAME_CONTRACT_STATUS').distinct().rdd.flatMap(lambda x: x).collect()

# Aplicando a função de tratamento de palavra a cada valor único.
nova_lista = [tratamento_palavra(status) for status in status]

# Exibindo a lista resultante.
print(nova_lista)

['APPROVED', 'UNUSED_OFFER', 'CANCELED', 'REFUSED']


<br>
<br>
<br>
<br>

## **Criação de flags**

A criação de flags binárias 1 ou 0 será útil para nos auxiliar na visão temporal dos dados e na criação das variáveis.
<br>
<br>
Serão criadas flags para os últimos 3, 6, 12, 24 e 36 meses.

In [10]:
df_temp_01 = spark.sql('''

SELECT
  *,
  CASE
    WHEN DAYS_DECISION >=   -90 THEN 1 ELSE 0 END AS U3M,
  CASE
    WHEN DAYS_DECISION >=  -180 THEN 1 ELSE 0 END AS U6M,
  CASE
    WHEN DAYS_DECISION >=  -360 THEN 1 ELSE 0 END AS U12M,
  CASE
    WHEN DAYS_DECISION >=  -720 THEN 1 ELSE 0 END AS U24M,
  CASE
    WHEN DAYS_DECISION >= -1080 THEN 1 ELSE 0 END AS U36M
FROM
  dados
ORDER BY
  `SK_ID_PREV`;

''')
df_temp_01.createOrReplaceTempView('df_temp_01')
df_temp_01.show()

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+--------------------+---------------------+------------------------+----------------------+--------------------+-------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+----------------+----------------+--------------------+-----------+----------------+--------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+---+---+----+----+----+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|   RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|N

<br>
<br>
<br>
<br>

## **Criação de variáveis explicativas**

Vamos criar as variáveis explicativas com base nas variáveis já existentes, usando a sumarização para as janelas temporais criadas na etapa anterior. Já iremos também agregar na granularidade indivíduo usando o ID **`SK_ID_CURR`**, que é o mesmo ID encontrado na tabela de **application_train**.

### **Variáveis Explicativas Gerais**

In [11]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_01.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
                     'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
                     'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY', 'NFLAG_INSURED_ON_APPROVAL']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  # Verifica se a coluna atual não é uma coluna de flag.
  if not any(flag in coluna for flag in colunas_flags):
    for flag in colunas_flags:
      if 'DPD' in coluna or 'DAYS' in coluna or 'HOUR' in coluna:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MAX_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MIN_{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_02.show(20, False)

Quantidade Total de Variáveis Criadas: 300
Nomes das Variáveis Criadas: ['VL_TOT_AMT_ANNUITY_U3M_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U3M_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U3M_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U3M_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U6M_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U6M_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U6M_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U6M_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U12M_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U12M_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U12M_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U12M_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U24M_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U24M_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U24M_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U24M_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U36M_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U36M_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U36M_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U36M_PREVIOUS_APP', 'VL_TOT_AMT_APPLICATION_U3M_PREVIOUS_APP', 'VL_MED_AMT_APPLICATION_U3M_PREVIOUS_APP', 'VL_MAX_AMT_APPLICATION_U3M_PREVIOUS_APP', 'VL_MI

<br>

### **Variáveis Explicativas com base no "NAME_CONTRACT_STATUS"**

In [12]:
# Definindo as colunas para agregação.
colunas_agregacao_total = df_temp_01.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
                     'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
                     'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY', 'NFLAG_INSURED_ON_APPROVAL']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for categoria in nova_lista:
  for coluna in colunas_agregacao_total:
    # Verifica se a coluna atual não é uma coluna de flag.
    if not any(flag in coluna for flag in colunas_flags):
      for flag in colunas_flags:
        if 'DPD' in coluna or 'DAYS' in coluna or 'HOUR' in coluna:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
          expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
          expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
        else:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
          expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))
          expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_03 = df_temp_01.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_03.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_03.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_03.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_03.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_03.show(20, False)

Quantidade Total de Variáveis Criadas: 1200
Nomes das Variáveis Criadas: ['VL_TOT_AMT_ANNUITY_U3M_APPROVED_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U3M_APPROVED_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U3M_APPROVED_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U3M_APPROVED_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U6M_APPROVED_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U6M_APPROVED_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U6M_APPROVED_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U6M_APPROVED_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U12M_APPROVED_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U12M_APPROVED_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U12M_APPROVED_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U12M_APPROVED_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U24M_APPROVED_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U24M_APPROVED_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U24M_APPROVED_PREVIOUS_APP', 'VL_MIN_AMT_ANNUITY_U24M_APPROVED_PREVIOUS_APP', 'VL_TOT_AMT_ANNUITY_U36M_APPROVED_PREVIOUS_APP', 'VL_MED_AMT_ANNUITY_U36M_APPROVED_PREVIOUS_APP', 'VL_MAX_AMT_ANNUITY_U36M_APPROVED_PREVIOUS_APP', 'VL

<br>

## **Salvando as tabelas sumarizadas**

In [13]:
# Book com as variáveis explicativas gerais.
df_temp_02.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_01')

In [14]:
# Book com as variáveis explicativas com base no 'NAME_CONTRACT_STATUS'.
df_temp_03.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_02')

<br>
<br>
<br>
<br>

## **Sumarização dos books de variáveis (das outras bases)**

Após criarmos as variáveis a partir dos datasets **`POS_CASH_balance`**, **`installments_payments`** e **`credit_card_balance`** agora precisamos deixar elas na granularidade indivíduo através do ID **`SK_ID_CURR`**, que é o mesmo ID encontrado na tabela de **application_train**.
<br>
<br>
Primeiramente iremos ler as tabelas com as variáveis criadas, depois iremos fazer um join delas com a tabela de **`previous_application`** através do ID **`SK_ID_CURR`** e sumarizar as variáveis pelo ID **`SK_ID_CURR`** e flags temporais.

### **Leitura das tabelas com as variáveis criadas**

#### **book_pos_cash_balance**

In [15]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
pos_cash = spark.read.parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_pos_cash_balance')
pos_cash.createOrReplaceTempView('pos_cash')

# Excluindo as colunas de PK_DATA.
colunas_desejadas = [col for col in pos_cash.columns if col not in ['PK_DATREF', 'PK_DAT_PROC']]
pos_cash = pos_cash.select(colunas_desejadas)

# Quantidade de linhas.
num_rows = pos_cash.count()

# Quantidade de colunas.
num_columns = len(pos_cash.columns)

# Quantidade de IDs únicos.
num_id_prev = pos_cash.select(countDistinct('SK_ID_PREV').alias('num_id_prev')).collect()[0]['num_id_prev']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_PREV" únicos do DataFrame: {num_id_prev}')

Quantidade de linhas do DataFrame: 936325
Quantidade de colunas do DataFrame: 801
Quantidade de "SK_ID_PREV" únicos do DataFrame: 936325


#### **book_installments_payments**

In [16]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
installments = spark.read.parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_installments_payments')
installments.createOrReplaceTempView('installments')

# Excluindo as colunas de PK_DATA.
colunas_desejadas = [col for col in installments.columns if col not in ['PK_DATREF', 'PK_DAT_PROC']]
installments = installments.select(colunas_desejadas)

# Quantidade de linhas.
num_rows = installments.count()

# Quantidade de colunas.
num_columns = len(installments.columns)

# Quantidade de IDs únicos.
num_id_prev = installments.select(countDistinct('SK_ID_PREV').alias('num_id_prev')).collect()[0]['num_id_prev']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_PREV" únicos do DataFrame: {num_id_prev}')

Quantidade de linhas do DataFrame: 997752
Quantidade de colunas do DataFrame: 61
Quantidade de "SK_ID_PREV" únicos do DataFrame: 997752


#### **book_credit_card_balance**

In [17]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
credit_card = spark.read.parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_credit_card_balance')
credit_card.createOrReplaceTempView('credit_card')

# Excluindo as colunas de PK_DATA.
colunas_desejadas = [col for col in credit_card.columns if col not in ['PK_DATREF', 'PK_DAT_PROC']]
credit_card = credit_card.select(colunas_desejadas)

# Quantidade de linhas.
num_rows = credit_card.count()

# Quantidade de colunas.
num_columns = len(credit_card.columns)

# Quantidade de IDs únicos.
num_id_prev = credit_card.select(countDistinct('SK_ID_PREV').alias('num_id_prev')).collect()[0]['num_id_prev']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_PREV" únicos do DataFrame: {num_id_prev}')

Quantidade de linhas do DataFrame: 104307
Quantidade de colunas do DataFrame: 381
Quantidade de "SK_ID_PREV" únicos do DataFrame: 104307


<br>

### **Join das tabelas**

In [18]:
# Selecionando apenas as colunas desejadas do DataFrame df_temp_01
df_temp_01_selected = df_temp_01.select('SK_ID_PREV', 'SK_ID_CURR', 'U3M', 'U6M', 'U12M', 'U24M', 'U36M')

# Realizando o join das tabelas usando apenas as colunas selecionadas de df_temp_01
df_temp_04 = df_temp_01_selected.join(pos_cash, 'SK_ID_PREV', how='left') \
                                .join(installments, 'SK_ID_PREV', how='left') \
                                .join(credit_card, 'SK_ID_PREV', how='left')

# Quantidade de linhas.
num_rows = df_temp_04.count()

# Quantidade de colunas.
num_columns = len(df_temp_04.columns)

# Quantidade de IDs únicos.
num_id_curr = df_temp_04.select(countDistinct('SK_ID_curr').alias('num_id_curr')).collect()[0]['num_id_curr']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_CURR" únicos do DataFrame: {num_id_curr}')

Quantidade de linhas do DataFrame: 1670214
Quantidade de colunas do DataFrame: 1247
Quantidade de "SK_ID_CURR" únicos do DataFrame: 338857


<br>

### **Sumarização dos dados (por janela de tempo)**



**Últimos 3 meses (U3M)**

In [19]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_04.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'U3M', 'U6M', 'U12M', 'U24M', 'U36M']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  for flag in colunas_flags:
    if 'TOT' in coluna:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MED' in coluna:
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MAX' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MIN' in coluna:
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_05 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_05.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_05.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_05.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_05.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1240
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH_U3M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH_U3M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH_U3M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH_U3M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH_U3M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U36M_POS_CASH_U

In [20]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_05.show(20, False)

In [21]:
# Salvando a tabela sumarizada.
df_temp_05.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_03')

<br>

**Últimos 6 meses (U6M)**

In [22]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_04.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'U3M', 'U6M', 'U12M', 'U24M', 'U36M']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U6M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  for flag in colunas_flags:
    if 'TOT' in coluna:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MED' in coluna:
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MAX' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MIN' in coluna:
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_06 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_06.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_06.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_06.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_06.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1240
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH_U6M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH_U6M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH_U6M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH_U6M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH_U6M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U36M_POS_CASH_U

In [23]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_06.show(20, False)

In [24]:
# Salvando a tabela sumarizada.
df_temp_06.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_04')

<br>

**Últimos 12 meses (U12M)**

In [25]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_04.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'U3M', 'U6M', 'U12M', 'U24M', 'U36M']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U12M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  for flag in colunas_flags:
    if 'TOT' in coluna:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MED' in coluna:
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MAX' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MIN' in coluna:
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_07 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_07.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_07.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_07.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_07.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1240
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH_U12M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH_U12M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH_U12M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH_U12M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH_U12M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT

In [26]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_07.show(20, False)

In [27]:
# Salvando a tabela sumarizada.
df_temp_07.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_05')

<br>

**Últimos 24 meses (U24M)**

In [28]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_04.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'U3M', 'U6M', 'U12M', 'U24M', 'U36M']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U24M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  for flag in colunas_flags:
    if 'TOT' in coluna:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MED' in coluna:
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MAX' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MIN' in coluna:
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_08 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_08.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_08.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_08.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_08.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1240
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH_U24M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH_U24M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH_U24M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH_U24M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH_U24M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT

In [29]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_08.show(20, False)

In [30]:
# Salvando a tabela sumarizada.
df_temp_08.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_06')

<br>

**Últimos 36 meses (U36M)**

In [31]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_04.columns

variaveis_remover = ['SK_ID_PREV', 'SK_ID_CURR', 'U3M', 'U6M', 'U12M', 'U24M', 'U36M']

colunas_agregacao_total = [col for col in colunas_agregacao_total if col not in variaveis_remover]

# Defindo a lista de colunas de flags.
colunas_flags = ['U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  for flag in colunas_flags:
    if 'TOT' in coluna:
      expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MED' in coluna:
      expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MAX' in coluna:
      expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))
    elif 'MIN' in coluna:
      expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'{coluna.upper()}_{flag.upper()}_PREVIOUS_APP'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_09 = df_temp_04.groupBy('SK_ID_CURR').agg(*expressoes_agregacao).orderBy('SK_ID_CURR')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_09.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_09.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_09.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_09.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

Quantidade Total de Variáveis Criadas: 1240
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH_U36M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH_U36M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH_U36M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH_U36M_PREVIOUS_APP', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH_U36M_PREVIOUS_APP', 'VL_TOT_CNT_INSTALMENT

In [32]:
# Mostrando o novo DataFrame com as variáveis criadas.
#df_temp_09.show(20, False)

In [34]:
# Salvando a tabela sumarizada.
df_temp_09.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_previous_application_07')